# Data Save

### Library Import

In [ ]:
import os
import cv2
import h5py
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from torchvision.transforms import transforms

### 데이터의 경로 입력

In [33]:
DATA_PATH = "dataset\\"

### train, test, val 폴더 경로 구분

In [34]:
file_path = dict()
for path in os.listdir(DATA_PATH):
    if "train" in path:
        file_path["train_path"] = os.path.join(DATA_PATH, path)
    elif "val" in path:
        file_path["val_path"] = os.path.join(DATA_PATH, path)
    else:
        file_path["test_path"] = os.path.join(DATA_PATH, path)

In [35]:
file_path

{'test_path': 'dataset\\test',
 'train_path': 'dataset\\train',
 'val_path': 'dataset\\val'}

### label 목록 저장

In [36]:
label_list = [label for label in os.listdir(file_path["train_path"])]
label_list

['battery',
 'biological',
 'cardboard',
 'clothes',
 'glass',
 'metal',
 'paper',
 'plastic',
 'shoes',
 'trash']

In [ ]:
train_img = []      # 파일명 저장
train_label = []    # label 값 저장

for label in label_list:
    for img_path in os.listdir(os.path.join(file_path["train_path"], label)):
            train_img.append(img_path)
            train_label.append(img_path.split("_")[0])
    for img_path in os.listdir(os.path.join(file_path['val_path'], label)):
          train_img.append(img_path)
          train_label.append(img_path.split("_")[0])

In [42]:
train_df = pd.DataFrame({
    'file_name' : train_img,
    'label': train_label
})
train_df

,file_name,label
0,battery_1.jpg,battery
1,battery_10.jpg,battery
2,battery_100.jpg,battery
3,battery_101.jpg,battery
4,battery_102.jpg,battery
...,...,...
8995,trash_767.jpg,trash
8996,trash_786.jpg,trash
8997,trash_788.jpg,trash
8998,trash_81.jpg,trash


In [ ]:
train_data = []
test_data = []

for img in os.path.join(file_path['train_path'])

In [ ]:
transforms.Resize(size=224)